<!--NAVIGATION-->
< [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Contents](Index.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) >

# 3.9 累计与分组

在对较大的数据进行分析时，一项基本的工作就是有效的数据累计（summarization）：计算累计指标，如``sum()``, ``mean()``, ``median()``, ``min()``,和 ``max()``，其中每一个指标都呈现了大数据集的特征。在这一节中，我们将探索 Pandas 的累计功能，从类似前面NumPy数组中的简单操作，到基于``groupby``实现的复杂操作。

我们将继续使用``display``函数方便显示。 

In [22]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 3.9.1 行星数据

我们将通过Seaborn程序库（http://seaborn.pydata.org） 用一份行星数据来进行演示，其中包含天文学家观测到的围绕恒星运转的行星数据（通常简称为太阳系外行星或外行星）。行星数据可以直接通过Seaborn下载：

In [23]:
import seaborn as sns
#planets = sns.load_dataset('planets')
#planets.to_csv('data/planets_data.csv',index=False) #为了更方便的查看下载的数据，我将它保存为了.csv数据
planets = pd.read_csv('data/planets_data.csv')
planets.shape

(1035, 6)

In [24]:
planets.head()#显示头几行数据，默认显示前5行；planets.head(10)显示前10行

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [25]:
help(planets.head)

Help on method head in module pandas.core.generic:

head(n=5) method of pandas.core.frame.DataFrame instance
    Return the first `n` rows.
    
    This function returns the first `n` rows for the object based
    on position. It is useful for quickly testing if your object
    has the right type of data in it.
    
    Parameters
    ----------
    n : int, default 5
        Number of rows to select.
    
    Returns
    -------
    obj_head : same type as caller
        The first `n` rows of the caller object.
    
    See Also
    --------
    DataFrame.tail: Returns the last `n` rows.
    
    Examples
    --------
    >>> df = pd.DataFrame({'animal':['alligator', 'bee', 'falcon', 'lion',
    ...                    'monkey', 'parrot', 'shark', 'whale', 'zebra']})
    >>> df
          animal
    0  alligator
    1        bee
    2     falcon
    3       lion
    4     monkey
    5     parrot
    6      shark
    7      whale
    8      zebra
    
    Viewing the first 5 lines
    


数据中包含了截至2014年已被发现的一千多颗系外行星的资料。

百度百科：太阳系外行星（简称系外行星；英语：extrasolar planet或exoplanet）泛指在太阳系以外的行星。历史上天文学家一般相信在太阳系以外存在着其它行星，然而它们的普遍程度和性质则是一个谜。直至1990年代人类才首次确认系外行星的存在，而自2002年起每年都有超过20个新发现的系外行星。现时估计不少于10%类似太阳的恒星都有其行星。

## 3.9.2 Pandas的简单累计功能

之前我们介绍过NumPy数组的一些数据累计指标（详情请参见2.4节）。与一维NumPy数组相同，Pandas的``Series``的累计函数也会返回一个统计值：

In [26]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [27]:
ser.sum()

2.811925491708157

In [28]:
ser.mean()

0.5623850983416314

``DataFrame``的累计函数默认对**每列**进行统计：

In [29]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [30]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

设置``axis``参数，你就可以对每一行进行统计了：

In [31]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas的``Series``和``DataFrame``支持所有2.4节中介绍的常用累计函数。另外，还有一个非常方便的``describe()``方法可以计算每一列的若干常用统计值。让我们在行星数据上试验一下，首先丢弃有缺失值的行：

In [32]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


这是一种理解数据集所有统计属性的有效方法。例如，从年份year列中可以看出， 1989年首次发现外行星，而且一半的已知外行星都是在2010年及以后的年份被发现的。这主要得益于**开普勒计划**——一个通过激光望远镜发现恒星周围椭圆轨道行星的太空计划。

Pandas内置的一些累计方法如下表所示。

| 指标                     | 描述                                   |
|--------------------------|----------------------------------------|
| ``count()``              | 计数项                                 |
| ``first()``, ``last()``  | 第一项与最后一项                       |
| ``mean()``, ``median()`` | 均值与中位数                           |
| ``min()``, ``max()``     | 最小值与最大值                         |
| ``std()``, ``var()``     | 标准差与方差                           |
| ``mad()``                | 均值绝对偏差（mean absolute deviation）|
| ``prod()``               | 所有项乘积                             |
| ``sum()``                | 所有项求和                             |

``DataFrame``和``Series``对象支持以上所有方法。

但若想深入理解数据，仅仅依靠累计函数是远远不够的。**数据累计的下一级别是``groupby``操作，它可以让你快速、有效地计算数据各子集的累计值。**

## 3.9.3 GroupBy：分割、应用和组合

简单的累计方法可以让我们对数据集有一个笼统的认识，但是我们经常还需要对某些标签或索引的局部进行累计分析，这时就需要用到``groupby``了。

虽然“分组”（group by）这个名字是借用 SQL 数据库语言的命令，但其理念引用发明R语言frame的Hadley Wickham的观点可能更合适：**分割（split）、应用（apply）和组合（combine）**。

### 1. 分割、应用和组合

一个经典分割-应用-组合操作示例如下图所示，其中“apply”的是一个求和函数。

![](figures/03.08-split-apply-combine.png)
[figure source in Appendix](06.00-Figure-Code.ipynb#Split-Apply-Combine)

上图清晰地描述了``groupby``的过程。

- 分割步骤将``DataFrame``按照指定的键分割成若干组。

- 应用步骤对每个组应用函数，通常是累计、转换或过滤函数。

- 组合步骤将每一组的结果合并成一个输出数组。

虽然我们也可以通过前面介绍的一系列的掩码、累计与合并操作来实现，但是意识到中间分割过程不需要显式地暴露出来这一点十分重要。而且``GroupBy``（经常）只需要一行代码，就可以计算每组的和、均值、计数、最小值以及其他累计值。**``GroupBy``的用处就是将这些步骤进行抽象：用户不需要知道在底层如何计算，只要把操作看成一个整体就够了。**

用Pandas进行上图所示的计算作为具体的示例。从创建输入``DataFrame``开始：

In [33]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


我们可以用``DataFrame``的``groupby()``方法进行绝大多数常见的分割-应用-组合操作，将需要分组的列名传进去即可：

In [34]:
df.groupby('key')

需要注意的是，这里的返回值不是一个``DataFrame``对象，而是一个``DataFrameGroupBy``对象。这个对象的魔力在于，你可以将它看成是一种**特殊形式**的 ``DataFrame``，里面**隐藏着若干组数据**，但是在没有应用累计函数之前不会计算。这种“延迟计算”（lazy evaluation）的方法使得大多数常见的累计操作可以通过一种对用户而言几乎是透明的（感觉操作仿佛不存在）方式非常高效地实现。

为了得到这个结果，可以对``DataFrameGroupBy``对象应用累计函数，它会完成相应的应用/组合步骤并生成结果：

In [35]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


``sum()``只是众多可用方法中的一个。你可以用Pandas或NumPy的任意一种累计函数，也可以用任意有效的``DataFrame``对象。下面就会介绍。

### 2. GroupBy对象

``GroupBy``对象是一种非常灵活的抽象类型。在大多数场景中，你可以将它看成是``DataFrame``的集合，在底层解决所有难题。让我们用行星数据来做一些演示。

``GroupBy``中最重要的操作可能就是*aggregate、filter、transform和apply（累计、过滤、转换、应用）*了，后文将详细介绍这些内容，现在先来介绍一些``GroupBy``的基本操作方法。

#### 基本操作之按列取值

``GroupBy``对象与``DataFrame``一样，也支持按列取值，并返回一个修改过的``GroupBy``对象，例如：

In [36]:
planets.groupby('method')

In [37]:
planets.groupby('method')['orbital_period']

这里从原来的``DataFrame``中取某个列名作为一个``Series``组。与``GroupBy``对象一样，直到我们运行累计函数，才会开始计算：

In [38]:
planets.groupby('method')['orbital_period'].median() #表示对method分组后，求每组中orbital_period的中位数

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

这样就可以获得不同方法下所有行星公转周期（按天计算）的中位数。

#### 基本操作之按组迭代

``GroupBy``对象支持直接按组进行迭代，返回的每一组都是``Series``或``DataFrame``：

In [39]:
for (method, group) in planets.groupby('method'):
    print("{0:40s} shape={1}".format(method, group.shape))

Astrometry                               shape=(2, 6)
Eclipse Timing Variations                shape=(9, 6)
Imaging                                  shape=(38, 6)
Microlensing                             shape=(23, 6)
Orbital Brightness Modulation            shape=(3, 6)
Pulsar Timing                            shape=(5, 6)
Pulsation Timing Variations              shape=(1, 6)
Radial Velocity                          shape=(553, 6)
Transit                                  shape=(397, 6)
Transit Timing Variations                shape=(4, 6)


尽管通常还是使用内置的``apply``功能速度更快，但这种方式在手动处理某些问题时非常有用，后面会详细介绍。

#### 基本操作之调用方法

借助Python类的魔力，可以让任何不由``GroupBy``对象直接实现的方法直接应用到每一组，无论是``DataFrame``还是``Series``对象都同样适用。

例如，你可以用``DataFrame``的``describe()``方法进行累计，对每一组数据进行描述性统计：

In [40]:
#pd.set_option('display.max_rows', None)  # 设置显示最大行，如果你的jupyter运行时不显示所有的行，而是用...表示那些没有显示的行时，使用此代码
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


这张表可以帮助我们对数据有更深刻的认识，例如大多数行星都是通过Radial Velocity和 Transit方法发现的，而且后者在近十年变得越来越普遍（得益于更新、更精确的望远镜）。最新的Transit Timing Variation和Orbital Brightness Modulation方法在2011年之后才有新的发现。

这只是演示Pandas调用方法的示例之一。方法首先会应用到每组数据上，然后结果由``GroupBy``组合后返回。另外，任意``DataFrame``/``Series``的方法都可以由 ``GroupBy``方法调用，从而实现非常灵活强大的操作。

### 3. 累计、过滤、转换和应用

虽然前面的章节只重点介绍了组合操作，但是还有许多操作没有介绍，尤其是``GroupBy``对象的``aggregate()``, ``filter()``, ``transform()``和 ``apply()``方法，在数据组合之前实现了大量高效的操作。

为了方便后面内容的演示，使用下面这个``DataFrame``：

In [41]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### (1) 累计

我们目前比较熟悉的``GroupBy``累计方法只有``sum()``和``median()``之类的简单函数，但是``aggregate()``其实可以支持更复杂的操作，比如字符串、函数或者函数列表，并且能一次性计算所有累计值。

下面来快速演示一个例子：

In [42]:
df.groupby('key').aggregate(['min', np.median, max])#支持函数名字符串、numpy函数、python内置函数

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

另一种用法就是通过Python字典指定不同列需要累计的函数：

In [43]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### (2) 过滤

过滤操作可以让你按照分组的属性丢弃若干数据。例如，我们可能只需要保留标准差超过某个阈值的组：

In [45]:
def filter_func(x):
    # x是一个分组数据的DataFrame
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

In [46]:
help(df.groupby('key').filter)

Help on method filter in module pandas.core.groupby.generic:

filter(func, dropna=True, *args, **kwargs) method of pandas.core.groupby.generic.DataFrameGroupBy instance
    Return a copy of a DataFrame excluding elements from groups that
    do not satisfy the boolean criterion specified by func.
    
    Parameters
    ----------
    f : function
        Function to apply to each subframe. Should return True or False.
    dropna : Drop groups that do not pass the filter. True by default;
        if False, groups that evaluate False are filled with NaNs.
    
    Returns
    -------
    filtered : DataFrame
    
    Notes
    -----
    Each subframe is endowed the attribute 'name' in case you need to know
    which group you are working on.
    
    Examples
    --------
    >>> df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
    ...                           'foo', 'bar'],
    ...                    'B' : [1, 2, 3, 4, 5, 6],
    ...                    'C' : [2.0, 5., 8., 1., 2.,

``filter()``函数会返回一个布尔值，表示每个组是否通过过滤。由于A组'data2'列的标准差不大于4，所以被丢弃了。

#### (3) 转换

累计操作返回的是对组内全量数据缩减过的结果，而转换操作会返回一个新的全量数据。数据经过转换之后，其形状与原来的输入数据是一样的。常见的例子就是将每一组的样本数据减去各组的均值，实现数据标准化：

In [25]:
df.groupby('key').transform(lambda x: x - x.mean())#对每个分组操作
#lambda 函数可以接收任意多个参数 (包括可选参数) 并且返回单个表达式的值。
#lambda函数的格式：冒号前是参数，可以有多个，用逗号隔开，冒号右边的为表达式。

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### apply() 方法

``apply()``方法让你可以在每个组上应用任意方法。这个函数输入一个``DataFrame``，返回一个Pandas对象（``DataFrame``或``Series``）或一个标量（``scalar``，单个数值）。组合操作会适应返回结果类型。

下面的例子就是用``apply()``方法将第一列数据以第二列的和为基数进行标准化：

In [28]:
def norm_by_data2(x):
    # x是一个分组数据的DataFrame
    x['data1'] = x['data1'] / x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)") #对每个分组做

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


``GroupBy``里的``apply()``方法非常灵活，唯一需要注意的地方是它总是输入分组数据的``DataFrame``，返回Pandas对象或标量。具体如何选择需要视情况而定。

### 5. 分组案例

通过下例中的几行Python代码，我们就可以运用上述知识，获取不同方法和不同年份发现的行星数量：

In [47]:
decade = 10 * (planets['year'] // 10)#//表示求整除的商
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum()#两层分组

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

In [48]:
planets.groupby(['method', decade])['number'].sum().unstack()

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,NaN,NaN,NaN,2.0
Eclipse Timing Variations,NaN,NaN,5.0,10.0
Imaging,NaN,NaN,29.0,21.0
Microlensing,NaN,NaN,12.0,15.0
Orbital Brightness Modulation,NaN,NaN,NaN,5.0
Pulsar Timing,NaN,9.0,1.0,1.0
Pulsation Timing Variations,NaN,NaN,1.0,NaN
Radial Velocity,1.0,52.0,475.0,424.0
Transit,NaN,NaN,64.0,712.0


In [49]:
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


此例足以展现GroupBy在探索真实数据集时快速组合多种操作的能力——只用寥寥几行代码，就可以让我们立即对过去几十年里不同年代的行星发现方法有一个大概的了解。

我建议你花点时间分析这几行代码，确保自己真正理解了每一行代码对结果产生了怎样的影响。虽然这个例子的确有点儿复杂，但是理解这几行代码的含义可以帮你掌握分析类似数据的方法。

<!--NAVIGATION-->
< [Combining Datasets: Merge and Join](03.07-Merge-and-Join.ipynb) | [Contents](Index.ipynb) | [Pivot Tables](03.09-Pivot-Tables.ipynb) >